In [1]:
import autogen
from prompt_templates import populatePrompts
from auto_functions import CustomGroupChat, is_termination_msg
import random
from typing import List, Dict
from autogen.agentchat.groupchat import GroupChat
from autogen.agentchat.agent import Agent
from autogen.agentchat.assistant_agent import AssistantAgent
from mockdata import get_data
# from autoapi import *



config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",  # or OAI_CONFIG_LIST.json if file extension is added
    filter_dict={
        "model": {
            "gpt-4-1106-preview",
            "gpt-3.5-turbo-1106",
            "gpt-3.5-tubro",
            "gpt-4-vision-preview"
        }
    }
)

gpt_config = {
    "cache_seed": 42,
    "temperature": 0.4,
    "config_list":config_list,
    "timeout":120,
    "stream": True,
}


In [2]:
#retrieve data
promptKeywords = get_data()
# promptKeywords
prompts = populatePrompts(**promptKeywords)
globals().update(prompts)

In [3]:

#user proxy agents

Human = autogen.UserProxyAgent(
    name = "Human_Student",
    system_message = human_prompt,
    code_execution_config=False,
    human_input_mode= 'ALWAYS'
)

terminator = autogen.UserProxyAgent(
    name = "Next_Question_Initiator",
    system_message="A Human admin. which will terminate the conversation whenever the Human_Student is ready to be asked the next question.",
    code_execution_config=False,
    # is_termination_msg=next_question_terminator(),
    human_input_mode="NEVER",
)

#assistant agents

AI_teacher = autogen.AssistantAgent(
    name = "AI_Teacher",
    llm_config=gpt_config,
    system_message= AI_teacher_prompt,
)

AI_student = autogen.AssistantAgent(
    name = "AI_student",
    llm_config=gpt_config,
    system_message= AI_student_prompt
)

# AI_quality_control = autogen.AssistantAgent(
#     name = "AI_Quality_Control",
#     llm_config= gpt_config,
#     system_message= AI_quality_control_prompt
# )

# critic = autogen.AssistantAgent(
#     name="Critic",
#     system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
#     llm_config=gpt4_config,
# )

AI_grader = autogen.AssistantAgent(
    name = "Grader",
    llm_config=gpt_config,
    system_message = grader_prompt,
)

theory_discusser = autogen.AssistantAgent(
    name = "Theory_Discusser",
    llm_config=gpt_config,
    system_message = theory_discusser_prompt)

clarifying_AI = autogen.AssistantAgent(
    name = "Question_Clarifier",
    llm_config=gpt_config,
    system_message = clarifying_AI_prompt)


groupchat = autogen.GroupChat(agents = [Human, AI_teacher, AI_student, AI_grader, theory_discusser, clarifying_AI],messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt_config)



In [8]:
Human.message

AttributeError: 'UserProxyAgent' object has no attribute 'message'

In [5]:
Human.initiate_chat(
    manager,
    message=groupchat_initial_prompt,
)

Human_Student (to chat_manager):


   You are all working together as a team act as an interactive question and answer system for Human_student. Your goal is to work together to create a question for a human student (Human_student) to answer, to grade that answer, and to provide feedback as necessary.

   The general flow of the conversation should be as follows:

   1. AI_Teacher should generate a question based on the provided information.
   The question should be sent to the human student for them to answer.
   2. wait for the Human_student to provide an answer
   3. AI_grader will take the Human_student's answer and grade it by using the AI_student's answer as a benchmark.
   4. Depending on the result of the grade, one of several things could happen.
    
     a. The AI_grader has graded the Human_student and their answer was correct (grade>=80%). In this case the Human_student will be congratulated and prompted about whether they would like to move onto the next question or if t

KeyboardInterrupt: 

In [10]:
print(autogen.ChatCompletion.logged_history)

None


In [ ]:
def print_messages(recipient, messages, sender, config):
    if "callback" in config and  config["callback"] is not None:
        callback = config["callback"]
        callback(sender, recipient, messages[-1])
    print(f"Messages sent to: {recipient.name} | num messages: {len(messages)}")
    return False, None  # required to ensure the agent communication flow continues

user_proxy.register_reply(
    [autogen.Agent, None],
    reply_func=print_messages,
    config={"callback": None},
)

assistant.register_reply(
    [autogen.Agent, None],
    reply_func=print_messages,
    config={"callback": None},
)

In [12]:
from transformers import GPT2Tokenizer

def count_tokens_in_chat(chat_history):
    # Initialize the tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    total_tokens = 0
    for message in chat_history.messages:
        # Assuming each message is a dictionary, and the text is under a key like 'content'
        message_text = message.get('content', '')  # Adjust 'content' based on your dictionary keys
        tokens = tokenizer.encode(message_text)
        total_tokens += len(tokens)

    return total_tokens

# Example usage
# Assuming 'groupchat' is your AutoGen GroupChat instance
total_tokens = count_tokens_in_chat(groupchat)
print(f"Total number of tokens in chat history: {total_tokens}")

Total number of tokens in chat history: 2986


In [14]:
groupchat.messages

[{'content': "\n   You are all working together as a team act as an interactive question and answer system for Human_student. Your goal is to work together to create a question for a human student (Human_student) to answer, to grade that answer, and to provide feedback as necessary.\n\n   The general flow of the conversation should be as follows:\n\n   1. AI_Teacher should generate a question based on the provided information.\n   The question should be sent to the human student for them to answer.\n   2. wait for the Human_student to provide an answer\n   3. AI_grader will take the Human_student's answer and grade it by using the AI_student's answer as a benchmark.\n   4. Depending on the result of the grade, one of several things could happen.\n    \n     a. The AI_grader has graded the Human_student and their answer was correct (grade>=80%). In this case the Human_student will be congratulated and prompted about whether they would like to move onto the next question or if they would